# SYSC4906 Assignment 3

**Group Name:** Haseeb and Abraham

**Student names:** Haseeb Khan, Abraham Srna

**Student numbers:** 101009713, 100997482


# Code to Train Your Method
_We will look at this, but will not run it when measuring your accuracy. Please structure your training code into logical steps, so that we can easily understand it_
## Step 1: Load the image dataset...

In [0]:
from keras.utils import to_categorical
from glob import glob
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from PIL import Image
from google.colab import drive
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing import image as kp_image

# Constants
NEW_IMAGE_LENGTH = 500
NEW_IMAGE_HEIGHT = 500
CONTENT_DRIVE_PATH = '/content/drive'
PATH_TO_IMAGES = CONTENT_DRIVE_PATH + "/My Drive/All_Images"
BUILDING_CODES = ["AA", "CB", "CT", "DT", "FH", "HP", "HS", "LB",
                  "MC", "ME", "ML", "PA", "RB", "RO", "SA", "TB"]
TRAINING_SIZE = 0.1

# Use one hot encoding to encode all of the building codes
numericCodes = range(0, len(BUILDING_CODES))
oneHotEncodedCodes = to_categorical(numericCodes)

# Mount the Google Drive containing all of the images
drive.mount(CONTENT_DRIVE_PATH)

# Get the paths for all the training and testing data
xTrain = []
yTrain = []
xTest = []
yTest = []

# Loop through all of the folders
for i in range(len(BUILDING_CODES)):
  # Get the building code and one hot encoded version
  code = BUILDING_CODES[i]
  oneHotCode = oneHotEncodedCodes[i]

  # Get the paths
  paths = glob(PATH_TO_IMAGES + "/" + code + "/*.jpg")

  # Split the paths for testing and training to create the hold-out set
  train, test = train_test_split(paths, test_size = TRAINING_SIZE)

  # Add the data to the corresponding lists
  xTrain += train
  yTrain += [oneHotCode] * len(train)
  xTest += test
  yTest += [oneHotCode] * len(test)

def cropAndResize(imgPath, newLen=NEW_IMAGE_LENGTH, newHeight=NEW_IMAGE_HEIGHT):
  """
  cropAndResize

  Opens the image at the given path. If the image is a rectangle,
  it is cropped to turn it into a square. The cropped square is then
  resized to the given sizes.

  @input  imgPath   Path to the image to crop and resize
  @input  newLen    The new length for the image
  @input  newHeight The new height for the image

  @return A PIL image containing the cropped and resized image
          (None if the image cannot be opened)
  """

  # Open the image if possible
  try:
    im = Image.open(imgPath)
  except:
    print("Error: Could not open image: %s" % (imgPath))
    return None

  # Get the current size
  width, height = im.size

  # Set the default cropped dimensions (square)
  left = 0
  top = 0
  right = width
  bot = height

  # Check if the image is portrait or landscape and if so,
  # get the new cropped dimensions
  if width > height:
    diff = (width - height) / 2

    left = diff
    top = 0
    right = width - diff
    bot = height
  elif width < height:
    diff = (height - width) / 2

    left = 0
    top = diff
    right = width
    bot = height - diff

  # Crop and resize the image
  im1 = im.crop((left, top, right, bot))
  im1 = im1.resize((newLen, newHeight))

  return(im1)

def formatImageData(listOfImagePaths, labels):
  """
  formatImageData

  Wrapper for the cropAndResize method to handle lists
  of image paths. Crops and resizes each image,
  converting each one to a Numpy array. If the path is
  invalid, then the corresponding image and label is removed.
  Returns the final list of labels and the cropped and resized
  images in a numpy array.

  @input  listOfImagePaths  The list of image paths to resize and crop
  @input  labels            The current list of labels

  @returns The formatted images in a numpy array and the list of
           corresponding labels
  """
  retImgs = []
  retLabels = []

  # Loop through all of the image paths
  for i in range(len(listOfImagePaths)):
    path = listOfImagePaths[i]
    label = labels[i]

    print("Formatting Image: %s" % (path))
    
    # Format the image
    resizedImage = cropAndResize(path)
    if resizedImage != None:
      # Convert the image to a numpy array and save the label
      retImgs.append(np.array(resizedImage))
      retLabels.append(label)

  # Convert the return list as a numpy array
  retImgs = np.asarray(retImgs)

  return(retImgs, retLabels)

# Format all of the training and testing data
xTrain, yTrain = formatImageData(xTrain, yTrain)
xTest, yTest = formatImageData(xTest, yTest)

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Formatting Image: /content/drive/My Drive/All_Images/AA/081_4.jpg
Formatting Image: /content/drive/My Drive/All_Images/AA/909-5.jpg
Formatting Image: /content/drive/My Drive/All_Images/AA/909-2.jpg
Formatting Image: /content/drive/My Drive/All_Images/AA/181_2.jpg
Formatting Image: /content/drive/My Drive/All_Images/AA/677-3.jpg
Formatting Image: /content/drive/My Drive/All_Images/AA/713_5.jpg
Formatting Image: /content/drive/My Drive/All_Images/AA/576_2.jpg
Fo

In [0]:
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input

# Set the training variables
NUM_CLASSES = 16

BATCH_SIZE = 16
EPOCHS = 100

TRAIN_DIR = "train"
TEST_DIR = "test"

# Set a new imput layer with the appropriate shape
inputLayer = Input(shape=(NEW_IMAGE_LENGTH, NEW_IMAGE_HEIGHT, 3))

# Load original model for transfer learning with a new input layer
# NOTE: We are not doing transfer learning fully as we are retraining the full
#       model, but we are still using the layers given by InceptionV3
baseModel = InceptionV3(weights = "imagenet",
                        include_top = False,
                        input_tensor = inputLayer)

# Add new layers for transfer learning
# Add a GlobalAveragePooling2D, Dropout, and Dense layer
x = baseModel.output
x = GlobalAveragePooling2D(name = "avg_pool")(x)
x = Dropout(0.5)(x)
preds = Dense(NUM_CLASSES, activation = "softmax")(x)
model = Model(inputs = baseModel.input, outputs = preds)

# Compile the new model
# Adam optimizer, categorical crossentropy as the loss,
# and accuracy as the metric
model.compile(optimizer = "Adam",
              loss = "categorical_crossentropy",
              metrics = ["accuracy"])

def createImageDataGenerator():
  """
  createImageDataGenerator

  Creates a new ImageDataGenerator and returns the created generator.

  @input  None

  @return The created generator
  """
  generator = ImageDataGenerator(preprocessing_function = preprocess_input,
                                 shear_range = 0.2, # Displace each point proportionally to simulate pictures of different proportions of buildings
                                 zoom_range = 0.2, # Zoom range to simulate taking pictures from different distances
                                 horizontal_flip = True, # Look from different angle
                                 width_shift_range = 0.2, # Simulate different proportions of a building
                                 height_shift_range = 0.2, # Simulate different proportions of a buildings
                                 rotation_range = 15, # Simulate taking pictures while holding camera at an angle
                                 vertical_flip = True, # Some training data images were backwards, so must trian for these situations
                                 fill_mode = 'reflect', # Carleton buildings normally have features repeated
                                 brightness_range = [0.5, 1.5], # Simulate sun/clouds and other weather conditions making image brighter or darker
                                 featurewise_center = True, # Treat each number in array as feature vector
                                 featurewise_std_normalization = True) # Normalization using feature vector
  
  return(generator)

# Create the generators
trainGenerator = createImageDataGenerator()
validationGenerator = createImageDataGenerator()

# Create the datagens using flow and the corresponding data
trainDatagen = trainGenerator.flow(xTrain,
                                   y = yTrain,
                                   batch_size = BATCH_SIZE)

validationDatagen = validationGenerator.flow(xTest,
                                             y = yTest,
                                             batch_size = BATCH_SIZE)

# Fit the generator
history = model.fit_generator(trainGenerator,
                              epochs = EPOCHS,
                              validation_data = validationGenerator)















87916544/87910968 [==============================] - 1s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


184/184 [==============================] - 209s 1s/step - loss: 2.8109 - acc: 0.0601 - val_loss: 14.3890 - val_acc: 0.0545
Epoch 2/100
184/184 [==============================] - 182s 989ms/step - loss: 2.7668 - acc: 0.0808 - val_loss: 6.8494 - val_acc: 0.0697
Epoch 3/100
184/184 [==============================] - 184s 998ms/step - loss: 2.7866 - acc: 0.0717 - val_loss: 2.7735 - val_acc: 0.0576
Epoch 4/100
184/184 [==============================] - 183s 993ms/step - loss: 2.7779 - acc: 0.0727 - val_loss: 8.3203 - val_acc: 0.0636
Epoch 5/100
184/184 [==============================] - 183s 993ms/step - loss: 2.7642 - acc: 0.0757 - val_loss: 6.3512 - val_acc: 0.0818
Epoch 6/100
184/184 [==============================] - 183s 993ms/step - loss: 2.7726 - acc: 0.0798 - val_loss: 2.7519 - val_acc: 0.0727
Epoch 7/100
184/184 [==============================] - 183s 993ms/step - loss: 2.7912 - acc: 0.0713 - val_loss: 2.8960 - val_acc: 0.0606
Epoch 8/100
184/184 [==============================] - 

In [0]:
def getMaxIndInList(listToParse):
  """
  Returns the index with the maximum value in the given list.

  @input  listToParse The list to check

  @return The index with the highest value
  """
  return(listToParse.index(max(listToParse)))

# Preprocess the test (holdout) set
x = xTest
x = preprocess_input(x)

# Predict all of the values in the test set
preds = model.predict(x)

# Initialuze the variables used to keep track of the correct count
# and the incorrect predictions
count = 0
matches = {}
for code in BUILDING_CODES:
  matches[code] = {}
  matches[code]["CORRECT"] = 0
  matches[code]["INCORRECT"] = {}
  for code2 in BUILDING_CODES:
    if code != code2:
      matches[code]["INCORRECT"][code2] = 0

# Loop through all of the predictions to check if they are correct
# and if incorrect, mark down what the incorrect value was
for i in range(len(preds)):
  # To get the prediction:
  #   For both the prediction and actual value:
  #     Covert the values to a list
  #     Get the maximum indices in the lists
  #     Get the final codes
  pred = preds[i].tolist()
  actual = yTest[i].tolist()

  predInd = getMaxIndInList(pred)
  actualInd = getMaxIndInList(actual)
  
  actualCode = BUILDING_CODES[actualInd]
  predCode = BUILDING_CODES[predInd]

  # Check if the indices match and iterate the corresponding values
  if (predInd == actualInd):
    matches[actualCode]["CORRECT"] += 1
    count += 1
  else:
    matches[actualCode]["INCORRECT"][predCode] += 1

# Print out the final information
maxCount = len(preds)
print("Result: %d/%d = %f" % (count, maxCount, count / maxCount))
for code in matches:
  print("Building Code: %s, Correct: %d" % (code, matches[code]["CORRECT"]))
  print("\tIncorrect predictions:")
  incorrect = matches[code]["INCORRECT"]
  for code2 in incorrect:
    if incorrect[code2] > 0:
      print("\t\tBuilding Code: %s, Times Mismatched: %d" % (code2, incorrect[code2]))

  


Result: 308/330 = 0.933333
Building Code: AA, Correct: 20
	Incorrect predictions:
Building Code: CB, Correct: 22
	Incorrect predictions:
Building Code: CT, Correct: 20
	Incorrect predictions:
		Building Code: CB, Times Mismatched: 1
Building Code: DT, Correct: 21
	Incorrect predictions:
		Building Code: MC, Times Mismatched: 1
Building Code: FH, Correct: 17
	Incorrect predictions:
		Building Code: TB, Times Mismatched: 1
Building Code: HP, Correct: 18
	Incorrect predictions:
		Building Code: AA, Times Mismatched: 1
		Building Code: FH, Times Mismatched: 1
Building Code: HS, Correct: 15
	Incorrect predictions:
		Building Code: CB, Times Mismatched: 3
Building Code: LB, Correct: 21
	Incorrect predictions:
Building Code: MC, Correct: 18
	Incorrect predictions:
		Building Code: LB, Times Mismatched: 2
		Building Code: SA, Times Mismatched: 1
Building Code: ME, Correct: 22
	Incorrect predictions:
Building Code: ML, Correct: 16
	Incorrect predictions:
		Building Code: CB, Times Mismatched: 2

##Step N: Save the model to file

In [0]:
# Whatever code is required to save your trained model to a file. 
# Should be re-loaded by prepareModel() during testing.
MODEL_FILE = "filename.model"

model.save(MODEL_FILE)